In [1]:
import sys
import pandas as pd
import numpy as np
import torch
from torch import Tensor

from matplotlib import pyplot as plt

sys.path.append("../")
from src.log import myLogger
from src.repository.data_repository import DataRepository
from src.checkpoint.checkpoint import Checkpoint
from src.metrics.jaccard import jaccard

pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 600)

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

In [2]:
logger = myLogger("../logs/temp.log", exp_id="e000", wdb_prj_id="temp", exp_config=None, use_wdb=False) 
dr = DataRepository(logger=logger, local_root_path="..")

2021-11-13 15:07:31,468 log.py               31   [INFO] [__init__] skip wandb init 


In [3]:
import pickle

prep_df = dr.load_preprocessed_df(
          dataset_name="train",                                                                                                                                                                                                                                   
          class_name="BaselineKernelPreprocessorV6",
          tokenizer_name="XLMRobertaTokenizerFast",
          max_length=400,
          pad_on_right=True,
          stride=135,
          split=False,
          lstrip=False,
          use_language_as_question=False,
          add_overflowing_batch_id=False,
    )

prep_df

2021-11-13 15:07:31,509 data_repository.py   253  [INFO] [load_preprocessed_df] loading data/preprocessed/train_BaselineKernelPreprocessorV6_XLMRobertaTokenizerFast_400_True_135_False_False_False_False.pkl ... 
2021-11-13 15:07:34,484 data_repository.py   260  [INFO] [load_preprocessed_df] done. 


,id,context,question,answer_text,answer_start,language,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,start_position,end_position,segmentation_position,is_contain_answer_text,max_successed_duplicated_elems_num_with
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,3,27,27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,3
1,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
2,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
3,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",3,1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
4,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",4,2,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,5,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,5
14184,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,5,351,352,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,5
14185,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,5,101,102,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,5
14186,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,20

#### load values

In [6]:
import gc
from collections import defaultdict
from tqdm.auto import tqdm

target_exp_ids = ["e059"]

checkpoints_info = {}

for target_exp_id in target_exp_ids:
    checkpoints_info[target_exp_id] = defaultdict(list)
    best_checkpoints = dr.best_checkpoint_filepaths(target_exp_id)
    for best_checkpoint in tqdm(best_checkpoints):
        exp_fold_checkpoint = dr.load_checkpoint_from_filepath(filepath_from_root=best_checkpoint, load_from_gcs=True, rm_local_after_load=True) 
        del exp_fold_checkpoint.model_state_dict
        del exp_fold_checkpoint.scheduler_state_dict
        del exp_fold_checkpoint.optimizer_state_dict
        gc.collect()
        checkpoints_info[target_exp_id]["val_ids"].extend(exp_fold_checkpoint.val_ids)
        checkpoints_info[target_exp_id]["val_start_logits"].extend(exp_fold_checkpoint.val_start_logits)
        checkpoints_info[target_exp_id]["val_end_logits"].extend(exp_fold_checkpoint.val_end_logits)
        del exp_fold_checkpoint
        gc.collect()

  0%|          | 0/5 [00:00<?, ?it/s]

2021-11-13 15:18:58,178 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e049/best_checkpoint/0_0_2.0046_0.6503.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e049/best_checkpoint/0_0_2.0046_0.6503.pkl 
2021-11-13 15:19:47,227 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-13 15:19:55,940 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e049/best_checkpoint/1_0_2.0477_0.7238.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e049/best_checkpoint/1_0_2.0477_0.7238.pkl 
2021-11-13 15:20:45,247 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-13 15:20:52,065 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e049/best_checkpoint/2_1_2.1020_0.6980.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e049/best_checkpoint/2_1_2.1020_0.6980.pkl 
2021-11-13 15:21:45,748 repository.py        176  [INFO] [__download_from_gcs] download

  0%|          | 0/5 [00:00<?, ?it/s]

2021-11-13 15:24:02,173 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e059/best_checkpoint/0_1_0.7868_0.6710.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e059/best_checkpoint/0_1_0.7868_0.6710.pkl 
2021-11-13 15:24:52,701 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-13 15:24:59,627 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e059/best_checkpoint/1_2_0.6392_0.7289.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e059/best_checkpoint/1_2_0.6392_0.7289.pkl 
2021-11-13 15:25:51,632 repository.py        176  [INFO] [__download_from_gcs] download done. 
2021-11-13 15:25:58,543 repository.py        167  [INFO] [__download_from_gcs] downloading data/checkpoint/e059/best_checkpoint/2_2_0.8982_0.7207.pkl from gs://kaggle-chaii-2021/../data/checkpoint/e059/best_checkpoint/2_2_0.8982_0.7207.pkl 
2021-11-13 15:26:47,905 repository.py        176  [INFO] [__download_from_gcs] download

#### merge values

In [43]:
checkpoints_info.keys()

dict_keys(['e059'])

In [44]:
# checkpoints_info["e049"]["val_ids"] == checkpoints_info["e059"]["val_ids"]

In [45]:
for target_exp_id in checkpoints_info.keys():
    checkpoints_info[target_exp_id]["val_start_logits"] = np.asarray(checkpoints_info[target_exp_id]["val_start_logits"])
    checkpoints_info[target_exp_id]["val_end_logits"] = np.asarray(checkpoints_info[target_exp_id]["val_end_logits"])
    
# for target_exp_id in checkpoints_info.keys():
#     checkpoints_info[target_exp_id]["start_logits"] = [np.asarray(logit) for logit in checkpoints_info[target_exp_id]["start_logits"]]
#     checkpoints_info[target_exp_id]["end_logits"] = [np.asarray(logit) for logit in checkpoints_info[target_exp_id]["end_logits"]]

In [46]:
ensembled_checkpoint_info = {
    "val_ids": None,
    "val_start_logits": None,
    "val_end_logits": None,
}

for target_exp_id in checkpoints_info.keys():
    if ensembled_checkpoint_info["val_ids"] is None:
        ensembled_checkpoint_info["val_ids"] = checkpoints_info[target_exp_id]["val_ids"]
    if ensembled_checkpoint_info["val_start_logits"] is None:
        ensembled_checkpoint_info["val_start_logits"] = checkpoints_info[target_exp_id]["val_start_logits"]
    else:
        ensembled_checkpoint_info["val_start_logits"] += checkpoints_info[target_exp_id]["val_start_logits"]
    if ensembled_checkpoint_info["val_end_logits"] is None:
        ensembled_checkpoint_info["val_end_logits"] = checkpoints_info[target_exp_id]["val_end_logits"]
    else:
        ensembled_checkpoint_info["val_end_logits"] += checkpoints_info[target_exp_id]["val_end_logits"]
        
        
ensembled_checkpoint_info["val_start_logits"] = [np.asarray(logit) for logit in checkpoints_info[target_exp_id]["val_start_logits"]]
ensembled_checkpoint_info["val_end_logits"] = [np.asarray(logit) for logit in checkpoints_info[target_exp_id]["val_end_logits"]]

In [47]:
checkpoint_df = pd.DataFrame()
checkpoint_df["id"] = ensembled_checkpoint_info["val_ids"]
checkpoint_df["start_logits"] = ensembled_checkpoint_info["val_start_logits"]
checkpoint_df["end_logits"] = ensembled_checkpoint_info["val_end_logits"]

checkpoint_df["overflowing_batch_id"] = None
bef_id = ""
overflowing_batch_id = 0
for i, row in checkpoint_df.iterrows():
    if str(row["id"]) != bef_id:
        overflowing_batch_id = 0
    checkpoint_df.loc[i, "overflowing_batch_id"] = overflowing_batch_id
    bef_id = row["id"]
    overflowing_batch_id += 1
checkpoint_df.head(10)

,id,start_logits,end_logits,overflowing_batch_id
0,903deec17,"[4.365806579589844, -7.971457481384277, -8.440...","[1.5394500494003296, -11.03594970703125, -10.2...",0
1,903deec17,"[6.402595520019531, -8.122644424438477, -8.349...","[4.5741286277771, -10.486711502075195, -10.039...",1
2,903deec17,"[6.430782318115234, -8.248629570007324, -8.225...","[4.6130146980285645, -10.36293888092041, -9.82...",2
3,903deec17,"[6.45772647857666, -8.249642372131348, -8.4133...","[4.593419551849365, -10.398869514465332, -10.0...",3
4,903deec17,"[6.577609062194824, -7.908515453338623, -8.251...","[4.7105393409729, -10.600852966308594, -10.001...",4
5,903deec17,"[6.293633460998535, -8.236004829406738, -8.517...","[4.575520992279053, -10.649038314819336, -10.1...",5
6,c29e29ab6,"[4.527185916900635, -8.384162902832031, -9.138...","[1.0810424089431763, -10.279254913330078, -9.4...",0
7,c29e29ab6,"[6.433323383331299, -7.929078102111816, -8.787...","[4.612836837768555, -9.79088020324707, -9.0493...",1
8,c29e29ab6,"[6.3108110427856445, -8.885916709899902, -9.01...","[4.393194198608398, -9.67039680480957, -8.9537...",2
9,c29e29ab6,"[6.305871486663818, -8.80740737915039, -9.1362...","[4.513009071350098, -9.805625915527344, -9.258...",3


In [49]:
prep_df = prep_df.merge(checkpoint_df, on=["id", "overflowing_batch_id"], how="left")
prep_df

,id,context,question,answer_text,answer_start,language,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,start_position,end_position,segmentation_position,is_contain_answer_text,max_successed_duplicated_elems_num_with,start_logits,end_logits
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,3,27,27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,3,"[4.365806579589844, -7.971457481384277, -8.440...","[1.5394500494003296, -11.03594970703125, -10.2..."
1,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3,"[6.402595520019531, -8.122644424438477, -8.349...","[4.5741286277771, -10.486711502075195, -10.039..."
2,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3,"[6.430782318115234, -8.248629570007324, -8.225...","[4.6130146980285645, -10.36293888092041, -9.82..."
3,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",3,1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3,"[6.45772647857666, -8.249642372131348, -8.4133...","[4.593419551849365, -10.398869514465332, -10.0..."
4,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",4,2,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3,"[6.577609062194824, -7.908515453338623, -8.251...","[4.7105393409729, -10.600852966308594, -10.001..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,5,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,5,"[5.825677394866943, -7.245992183685303, -9.023...","[3.8614253997802734, -11.019204139709473, -10...."
14184,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, 

In [88]:
from typing import List, Tuple

def get_context_part(
    offset_mapping: List[Tuple[int, int]], context: str
) -> str:
    s = 1_000_000_000
    e = 0
    for (offs, offe) in offset_mapping:
        if offs == -1:
            continue
        s = min(offs, s)
        e = max(offe, e)
    context_part = context[s:e]
    return s, e, context_part

def get_part_start_end_logit_score(start_char_index, end_char_index, offset_mapping, start_logit, end_logit):
    start_index = 0
    while offset_mapping[start_index][0] == -1:
        start_index += 1
    while offset_mapping[start_index][0] != -1 and offset_mapping[start_index][0] <= start_char_index:
        start_index += 1
    start_index -= 1
    end_index = len(offset_mapping) - 1
    while offset_mapping[end_index][1] == -1:
        end_index -= 1
    while offset_mapping[end_index][1] != -1 and offset_mapping[end_index][1] >= end_char_index:
        end_index -= 1
    end_index += 1
    score = start_logit[start_index] + end_logit[end_index]
    return score

In [93]:
# %debug
import json

for i, row in tqdm(prep_df.iterrows(), total=len(prep_df)):
    context_part_start_char_index, _, context_part = get_context_part(row["offset_mapping"], row["context"])
    char_index_score_dict = {}
    start_char_index = context_part.find(row["answer_text"])
    while start_char_index >= 0:
        score = get_part_start_end_logit_score(
            context_part_start_char_index+start_char_index,
            context_part_start_char_index+start_char_index+len(row["answer_text"]),
            row["offset_mapping"],
            row["start_logits"],
            row["end_logits"]
        )
        char_index_score_dict[context_part_start_char_index+start_char_index] = score

        context_part_start_char_index += (start_char_index+len(row["answer_text"]))
        context_part = context_part[start_char_index+len(row["answer_text"]):]
        start_char_index = context_part.find(row["answer_text"])
    prep_df.loc[i, "char_index_score_dict"] = json.dumps(char_index_score_dict)

  0%|          | 0/14188 [00:00<?, ?it/s]

In [105]:
prep_df.head(100)[["id", "question", "answer_text", "answer_start", "language", "overflowing_batch_id", "is_contain_answer_text", "char_index_score_dict"]]

,id,question,answer_text,answer_start,language,overflowing_batch_id,is_contain_answer_text,char_index_score_dict
0,903deec17,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,0,1,"{""53"": 11.66967248916626}"
1,903deec17,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,1,0,{}
2,903deec17,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,2,0,{}
3,903deec17,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,3,0,{}
4,903deec17,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,4,0,{}
5,903deec17,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,5,0,{}
6,d9841668c,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,0,0,{}
7,d9841668c,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,1,0,{}
8,d9841668c,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,2,1,"{""2358"": 2.94630229473114, ""2531"": -4.18077778..."
9,d9841668c,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil,3,1,"{""2653"": -17.986735343933105}"


In [133]:
def temp(x):
    y = json.loads(x)
    if len(y) == 0:
        return -1
    else:
        return max(y.values())
    
def temp2(x):
    y = json.loads(x)
    if len(y) == 0:
        return -1
    else:
        res_k = -1
        res_v = -1_000_000_000
        for k, v in y.items():
            if v > res_v:
                res_k = int(k)
                res_v = v
        return res_k
    
prep_df["max_score"] = prep_df.char_index_score_dict.apply(lambda x: temp(x))
prep_df["max_index"] = prep_df.char_index_score_dict.apply(lambda x: temp2(x))
prep_df

,id,context,question,answer_text,answer_start,language,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,start_position,end_position,segmentation_position,is_contain_answer_text,max_successed_duplicated_elems_num_with,start_logits,end_logits,char_index_score_dict,max_score,max_index
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,3,27,27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1,3,"[4.365806579589844, -7.971457481384277, -8.440...","[1.5394500494003296, -11.03594970703125, -10.2...","{""53"": 11.66967248916626}",11.669672,53
1,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3,"[6.402595520019531, -8.122644424438477, -8.349...","[4.5741286277771, -10.486711502075195, -10.039...",{},-1.000000,-1
2,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3,"[6.430782318115234, -8.248629570007324, -8.225...","[4.6130146980285645, -10.36293888092041, -9.82...",{},-1.000000,-1
3,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",3,1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3,"[6.45772647857666, -8.249642372131348, -8.4133...","[4.593419551849365, -10.398869514465332, -10.0...",{},-1.000000,-1
4,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",4,2,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,3,"[6.577609062194824, -7.908515453338623, -8.251...","[4.7105393409729, -10.600852966308594, -10.001...",{},-1.000000,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,5,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,5,"[5.825677394866943, -7.245992183685303, -9.023...","[3.8614253997802734, -11.019204139709473, -10....",{},-1.000000,-1
14184,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None

In [137]:
prep_df.query("max_score < 0").max_score.sort_values()

11170   -21.523368
11169   -21.163138
2266    -21.062973
2267    -20.808329
2246    -20.776046
           ...    
873      -0.041284
868      -0.034563
6648     -0.033757
12327    -0.025095
6670     -0.012419
Name: max_score, Length: 12559, dtype: float64

In [168]:
prep_df.groupby("id").max_score.max().sort_values().head(100)

id
2aa11c325   -7.544055
5efde6551   -2.071108
1dcc12497   -1.000000
1eacbc70f   -1.000000
86c77b886   -1.000000
d9da39b42   -1.000000
a4e9767b3   -1.000000
d637f9208   -1.000000
d5c4bbfd6   -1.000000
d54cb8e9d   -1.000000
0ddbdb4b5   -1.000000
3a4db1dda   -1.000000
0ceb8fc1d   -1.000000
3e9696e57   -1.000000
771ace3aa   -1.000000
e358d41e6   -1.000000
ea240e50a   -1.000000
56426ea3a   -1.000000
addf5671e   -1.000000
daddbeba7   -1.000000
a75af54c6   -1.000000
15e538650   -1.000000
422fd74bf   -1.000000
e24ada3ac   -1.000000
80452bc4d   -1.000000
e0090c270   -1.000000
8188e62a9   -1.000000
1a1654401   -1.000000
839bdc1fc   -1.000000
e7dce8a1c   -1.000000
1431f3af8   -1.000000
e7fce186a   -1.000000
de34ee0a0   -1.000000
1b8635229   -1.000000
dc4676ed4   -1.000000
ae61ff38a   -1.000000
cd1314476   -1.000000
3742b3620   -1.000000
76fc189e8   -1.000000
b743d8294   -1.000000
3fe627e15   -1.000000
33bf6b51c   -1.000000
4ca677224   -1.000000
01ae1bdd3   -1.000000
bc9f0d533   -1.000000
715d4fa

In [134]:
prep_df.groupby("id").max_index.nunique()

id
003332744    3
004938454    2
006bd6b9a    1
007f179a0    2
00ef07946    2
            ..
ff227273d    2
ff31c3eba    4
ff7cbbd6e    2
ff8d67175    2
ffda743a2    4
Name: max_index, Length: 1114, dtype: int64

In [177]:
prep_df.query("id == '8ec05af82'")

,id,context,question,answer_text,answer_start,language,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,duplicated_elems_num_with,start_position,end_position,segmentation_position,is_contain_answer_text,max_successed_duplicated_elems_num_with,start_logits,end_logits,char_index_score_dict,max_score,max_index
10221,8ec05af82,एंड्रयू हीथ लेजर (4 अप्रैल 1979 -22 जनवरी 2008...,एंड्रयू हीथ लेजर ने सबसे पहले किस फिल्म में का...,ब्लैकरोच,5792,hindi,"[0, 109582, 21906, 57655, 2093, 8531, 12129, 3...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,8,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,8,"[1.9461859464645386, -8.83395004272461, -9.380...","[0.266192227602005, -11.264069557189941, -11.0...",{},-1.00000,-1
10222,8ec05af82,एंड्रयू हीथ लेजर (4 अप्रैल 1979 -22 जनवरी 2008...,एंड्रयू हीथ लेजर ने सबसे पहले किस फिल्म में का...,ब्लैकरोच,5792,hindi,"[0, 109582, 21906, 57655, 2093, 8531, 12129, 3...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,6,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,8,"[6.021588325500488, -8.473138809204102, -9.088...","[4.6773600578308105, -10.497385025024414, -10....",{},-1.00000,-1
10223,8ec05af82,एंड्रयू हीथ लेजर (4 अप्रैल 1979 -22 जनवरी 2008...,एंड्रयू हीथ लेजर ने सबसे पहले किस फिल्म में का...,ब्लैकरोच,5792,hindi,"[0, 109582, 21906, 57655, 2093, 8531, 12129, 3...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,7,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,8,"[5.512380599975586, -8.36297607421875, -8.9878...","[4.383754730224609, -10.773411750793457, -10.4...",{},-1.00000,-1
10224,8ec05af82,एंड्रयू हीथ लेजर (4 अप्रैल 1979 -22 जनवरी 2008...,एंड्रयू हीथ लेजर ने सबसे पहले किस फिल्म में का...,ब्लैकरोच,5792,hindi,"[0, 109582, 21906, 57655, 2093, 8531, 12129, 3...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",3,6,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,8,"[5.572924613952637, -8.294378280639648, -8.875...","[4.322988033294678, -10.938131332397461, -10.3...",{},-1.00000,-1
10225,8ec05af82,एंड्रयू हीथ लेजर (4 अप्रैल 1979 -22 जनवरी 2008...,एंड्रयू हीथ लेजर ने सबसे पहले किस फिल्म में का...,ब्लैकरोच,5792,hindi,"[0, 109582, 21906, 57655, 2093, 8531, 12129, 3...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",4,5,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,8,"[5.667966842651367, -8.376611709594727, -8.707...","[4.423605918884277, -10.986933708190918, -10.1...",{},-1.00000,-1
10226,8ec05af82,एंड्रयू हीथ लेजर (4 अप्रैल 1979 -22 जनवरी 2008...,एंड्रयू हीथ लेजर ने सबसे पहले किस फिल्म में का...,ब्लैकरोच,5792,hindi,"[0, 109582, 21906, 57655, 2093, 8531, 12129, 3...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",5,6,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,8,"[5.965335845947266, -8.189024925231934, -8.564...","[4.670012474060059, -10.694731712341309, -9.93...",{},-1.00000,-1
10227,8ec05af82,एंड्रयू हीथ लेजर (4 अप्रैल 1979 -22 जनवरी 2008...,एंड्रयू हीथ लेजर ने सबसे पहले किस फिल्म में का...,ब्लैकरोच,5792,hindi,"[0, 109582, 21906, 57655, 2093, 8531, 12

In [179]:
row = prep_df.loc[10227]
print(row["is_contain_answer_text"])
print(row["question"])
print(row["answer_text"])
print(row["char_index_score_dict"])
print(row["offset_mapping"])
print(row["context"][5014:6244])

1
एंड्रयू हीथ लेजर ने सबसे पहले किस फिल्म में काम किया था?
ब्लैकरोच
{"5792": 0.13439003378152847}
[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (5014, 5017), (5017, 5020), (5020, 5023), (5023, 5029), (5029, 5033), (5033, 5040), (5040, 5042), (5042, 5043), (5043, 5045), (5045, 5048), (5048, 5054), (5054, 5055), (5055, 5057), (5057, 5067), (5067, 5070), (5070, 5073), (5073, 5074), (5074, 5077), (5077, 5079), (5079, 5080), (5080, 5083), (5083, 5086), (5086, 5090), (5090, 5095), (5095, 5096), (5096, 5097), (5097, 5101), (5101, 5105), (5105, 5109), (5109, 5113), (5114, 5120), (5122, 5127), (5127, 5130), (5130, 5134), (5135, 5138), (5138, 5143), (5143, 5146), (5146, 5150), (5150, 5154), (5154, 5161), (5161, 5167), (5167, 5170), (5170, 5178), (5178, 5183), (5183, 5186), (5186, 5190), (5190, 5193), (5193, 5195), (5195, 5198), (5198, 5204), (

In [176]:
row = prep_df.loc[5443]
print(row["is_contain_answer_text"])
print(row["question"])
print(row["answer_text"])
print(row["char_index_score_dict"])
print(row["offset_mapping"])
print(row["context"][14461:15614])

1
ताम्र एवं टीन के मिश्रधातु को क्या कहते हैं ?
कांसा
{"15541": -6.322589874267578}
[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (14461, 14464), (14464, 14465), (14465, 14472), (14472, 14477), (14477, 14481), (14481, 14484), (14484, 14487), (14487, 14489), (14489, 14491), (14491, 14494), (14494, 14496), (14496, 14497), (14497, 14499), (14499, 14501), (14501, 14505), (14505, 14509), (14509, 14513), (14513, 14515), (14515, 14518), (14518, 14523), (14523, 14528), (14528, 14532), (14532, 14533), (14533, 14538), (14538, 14541), (14541, 14543), (14543, 14547), (14547, 14550), (14550, 14556), (14556, 14559), (14559, 14563), (14563, 14567), (14567, 14570), (14570, 14575), (14575, 14576), (14576, 14580), (14580, 14582), (14582, 14587), (14587, 14593), (14593, 14596), (14596, 14599), (14599, 14605), (14605, 14608), (14608, 14609), (14609, 14612), (14612, 14613),

In [145]:
row["context"][4872-5:4872+len("कुनैन")]

'े ही कुनैन'

In [147]:
str(" हो। ")

' हो। '

In [128]:
row = prep_df.loc[5191]
print(row["question"])
print(row["answer_text"])
print(row["char_index_score_dict"])
print(row["offset_mapping"])
print(row["context"][0:1635])

எந்த ஆண்டில் இந்திய தேசிய காங்கிரஸ் நிறுவப்பட்டது?
1885
{"191": 12.250142097473145, "903": 9.944947481155396, "1320": 3.9632880091667175}
[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (0, 6), (6, 12), (12, 22), (22, 24), (24, 31), (31, 32), (32, 39), (39, 48), (48, 57), (57, 58), (58, 60), (60, 61), (61, 63), (63, 67), (67, 68), (68, 70), (70, 76), (76, 83), (83, 93), (93, 95), (95, 96), (96, 97), (97, 105), (105, 112), (112, 122), (122, 123), (123, 131), (131, 134), (134, 136), (136, 137), (137, 149), (149, 153), (153, 155), (155, 157), (157, 160), (160, 168), (168, 174), (174, 179), (179, 184), (184, 186), (186, 189), (189, 190), (190, 195), (195, 197), (197, 204), (204, 210), (210, 212), (212, 215), (215, 219), (219, 220), (220, 227), (227, 235), (235, 241), (241, 245), (245, 251), (251, 257), (257, 258), (258, 260), (260, 266), (266, 268), (268, 269), (269, 271), (271, 274), (274, 278), (278, 282),

In [132]:
row = prep_df.loc[5196]
print(row["question"])
print(row["answer_text"])
print(row["char_index_score_dict"])
print(row["offset_mapping"])
print(row["context"][1124:2539])

எந்த ஆண்டில் இந்திய தேசிய காங்கிரஸ் நிறுவப்பட்டது?
1885
{"6400": 7.515032052993774}
[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (4839, 4840), (4841, 4846), (4846, 4847), (4847, 4849), (4849, 4856), (4856, 4860), (4860, 4863), (4863, 4866), (4866, 4867), (4867, 4871), (4871, 4877), (4879, 4885), (4885, 4887), (4888, 4891), (4891, 4895), (4895, 4896), (4896, 4899), (4899, 4903), (4903, 4906), (4906, 4908), (4908, 4911), (4911, 4916), (4916, 4922), (4922, 4925), (4925, 4930), (4930, 4934), (4934, 4940), (4940, 4945), (4945, 4949), (4949, 4958), (4958, 4959), (4959, 4966), (4966, 4968), (4968, 4970), (4970, 4972), (4972, 4985), (4985, 4993), (4993, 4998), (4998, 5000), (5000, 5005), (5005, 5006), (5006, 5008), (5008, 5015), (5015, 5019), (5019, 5022), (5022, 5026), (5026, 5029), (5029, 5031), (5031, 5037), (5037, 5041), (5041, 5048), (5048, 5052), (5052, 5053), (5053, 5057), (5057, 5063), (5063, 5067), (

In [107]:
row = prep_df.loc[67]
print(row["offset_mapping"])
print(row["context"][:1317])

[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (0, 5), (5, 9), (9, 13), (13, 15), (15, 22), (22, 28), (28, 30), (30, 32), (32, 34), (34, 35), (35, 42), (42, 52), (52, 56), (56, 61), (61, 62), (62, 65), (65, 67), (67, 70), (70, 74), (74, 81), (81, 82), (82, 87), (87, 89), (89, 91), (91, 95), (95, 96), (96, 101), (101, 112), (112, 113), (113, 115), (115, 117), (117, 120), (120, 124), (124, 125), (125, 127), (127, 128), (128, 133), (133, 136), (136, 142), (142, 149), (149, 155), (155, 160), (160, 164), (164, 165), (165, 167), (167, 169), (169, 171), (171, 172), (172, 177), (177, 180), (180, 182), (182, 190), (190, 200), (200, 205), (205, 210), (210, 215), (215, 219), (219, 224), (224, 227), (227, 233), (233, 240), (240, 246), (246, 248), (248, 250), (250, 252), (252, 256), (256, 257), (257, 259), (259, 262), (262, 263), (263, 265), (265, 266), (266, 269), (269, 271), (271, 272), (272, 2

In [109]:
row = prep_df.loc[68]
print(row["offset_mapping"])
print(row["context"][878:2221])

[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (878, 880), (880, 885), (885, 886), (886, 889), (889, 891), (891, 893), (893, 895), (895, 896), (896, 899), (899, 901), (901, 908), (908, 911), (911, 914), (914, 917), (917, 919), (919, 921), (921, 926), (926, 932), (932, 935), (935, 936), (936, 939), (939, 941), (941, 945), (945, 953), (953, 954), (954, 957), (957, 961), (961, 963), (963, 966), (966, 967), (967, 968), (968, 971), (971, 975), (975, 978), (978, 979), (979, 982), (982, 986), (986, 990), (990, 991), (991, 996), (996, 998), (998, 1001), (1001, 1005), (1005, 1010), (1010, 1015), (1015, 1016), (1016, 1022), (1022, 1026), (1026, 1032), (1032, 1037), (1037, 1044), (1044, 1049), (1049, 1058), (1058, 1059), (1059, 1060), (1060, 1062), (1062, 1064), (1064, 1069), (1069, 1074), (1074, 1076), (1076, 1082), (1082, 1088), (1088, 1093), (1093, 1095), (1095, 1101), (1101, 1105), (1105, 1

In [114]:
row = prep_df.loc[47]
print(row["question"])
print(row["answer_text"])
print(row["char_index_score_dict"])
print(row["offset_mapping"])
print(row["context"][3473:4761])

திரிதடையங்களைப் பயன்படுத்திய முதல் நிறுவனம் எது?
IBM
{"4171": 3.6412532925605774, "4208": 5.769239664077759, "4352": -5.552682399749756, "4468": -9.261614799499512}
[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (3473, 3476), (3476, 3482), (3482, 3486), (3486, 3488), (3488, 3493), (3493, 3497), (3497, 3501), (3501, 3505), (3505, 3510), (3510, 3511), (3511, 3512), (3512, 3516), (3516, 3517), (3517, 3520), (3520, 3524), (3524, 3528), (3528, 3532), (3532, 3533), (3533, 3535), (3535, 3538), (3538, 3539), (3539, 3543), (3543, 3546), (3547, 3551), (3551, 3558), (3558, 3559), (3559, 3572), (3572, 3573), (3573, 3577), (3577, 3580), (3580, 3584), (3584, 3587), (3587, 3588), (3588, 3592), (3592, 3593), (3593, 3596), (3596, 3598), (3598, 3600), (3600, 3602), (3602, 3604), (3604, 3605), (3605, 3607), (3607, 3611), (3611, 3613), (3613, 3620), (3620, 3622), (3622, 3625), (3625, 3627), (3627, 3629)

In [119]:
row = prep_df.loc[14184]
print(row["question"])
print(row["answer_text"])
print(row["char_index_score_dict"])
print(row["offset_mapping"])
print(row["context"][817:4761])

लीजा रे की पहली फिल्म का नाम क्या था?
नेताजी
{"2017": 14.952022552490234}
[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1), (817, 818), (818, 821), (821, 824), (824, 829), (829, 832), (832, 834), (834, 837), (837, 841), (841, 843), (843, 848), (848, 852), (852, 856), (856, 861), (861, 865), (865, 869), (869, 874), (874, 882), (882, 883), (883, 888), (888, 889), (889, 894), (894, 897), (897, 901), (901, 904), (904, 906), (906, 909), (909, 912), (912, 914), (914, 916), (916, 919), (919, 924), (924, 927), (927, 931), (931, 934), (934, 940), (940, 945), (945, 950), (950, 954), (954, 955), (955, 958), (958, 963), (963, 967), (967, 970), (970, 980), (980, 986), (986, 989), (989, 995), (995, 998), (998, 1003), (1003, 1007), (1007, 1010), (1010, 1013), (1013, 1016), (1016, 1019), (1019, 1026), (1026, 1029), (1029, 1032), (1032, 1037), (1037, 1041), (1041, 1044), (1044, 1045), (1045, 1048), (1048, 1052), 

In [181]:
with open("../data/preprocessed/e059_prep_df.pkl", "wb") as fout:
    pickle.dump(prep_df, fout)

## prep df として作ってしまおう。
 - 1 text batch 内では argmax を使う
 - 0 より小さい場合は無視する
 - 1 id 内に一つもない場合は以下の 2 通りを扱う
     - argmax を使う
     - 無視する

In [98]:
prep_df.loc[8]["char_index_score_dict"]

'{"2358": 2.94630229473114, "2531": -4.1807777881622314, "2653": -3.6253418922424316}'

In [82]:
context_part_start_char_index

3023

In [72]:
start_char_index

163

In [76]:
get_context_part(row["offset_mapping"], row["context"])

(1713,
 3019,
 ' வாழ்ந்ததாக அறிஞர்கள் ஊகிக்கின்றனர்.[6]\nமேற்காணும் இந்த தகவல்கள், காளிதாசன் சமசுகிருத மொழியில் இயற்றிய காவியக் கவிதையான குமாரசம்பவம் எனும் நாடகக் கவிதையில், இமயமலைத் தொடர்களையும் காளிதாசரின் விரிவான விளக்கத்தையும் அடிப்படையாக கொண்டுள்ளது. மேலும், அவர் உஜ்ஜைனை ஆழமாக நேசித்த அன்பின் காட்சிகளாக, அவர் இயற்றிய மேகதூதம், மற்றும் இரகுவம்சம் எனும் காவியங்களில் மிகுந்த விளக்கங்கள் காணப்படுகின்றன.[7]\nகாசுமீர் பண்டிதரும், சமசுகிருத அறிஞருமான "லட்சுமி தார் கல்லா" (1891 - 1953) என்பவர், 1926 இல் காளிதாசாவின் பிறப்பு என்ற தலைப்பில் ஒரு புத்தகத்தை எழுதினார், அது காளிதாசனின் பிறப்பிடத்தை அவரது எழுத்துக்களில் அடிப்படையாகக் கொண்டது. மேலும் அவர் காளிதாசர் காசுமீரில் பிறந்தார் என்றும், ஆனால் தென்திசை நோக்கிச் சென்று உள்ளூர் ஆட்சியாளர்களின் ஆதரவைப் பெற முயன்றதாகவும் கூறப்படுகிறது.[8][9][10]\n உஜ்ஜைன் மற்றும் கலிங்காவில் அல்லாத, காசுமீரில் காணப்படும் தாவரம் மற்றும் விலங்கினங்களின் விவரம்: குங்குமப்பூ செடிகள், தேவதாரு மரங்கள், கத்தூரி மான் போன்றவை.\n காசுமீரில் பொதுவான புவியியல் அம்சங்களின்

In [77]:
row["answer_text"]

'காசுமீரில்'